In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
path = "/content/drive/MyDrive/Colab Notebooks/SMSSpamCollection"
df = pd.read_csv(path, sep='\t', names=['lable','message'])

In [3]:
df.head()

,lable,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
X=df['message']
y=df['lable']

In [5]:
y = pd.get_dummies(y,drop_first=True)['spam']

In [6]:
y = y.astype(int)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
)


In [8]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
X_train = X_train.tolist()
X_test = X_test.tolist()

In [10]:
train_encoding = tokenizer(X_train,truncation=True,padding=True,return_tensors="pt")
test_encoding = tokenizer(X_test,truncation=True,padding=True,return_tensors="pt")

In [11]:
import torch
y_train = torch.tensor(y_train.values)
y_test = torch.tensor(y_test.values)

In [12]:
train_encoding

{'input_ids': tensor([[  101,  7514,  2000,  ...,     0,     0,     0],
        [  101,  7592,  1012,  ...,     0,     0,     0],
        [  101,  2129,  2272,  ...,     0,     0,     0],
        ...,
        [  101, 10975,  7875,  ...,     0,     0,     0],
        [  101, 23961, 16644,  ...,     0,     0,     0],
        [  101,  2106,  2002,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [15]:
# ## Mental Picture of dataset
# tempdic = {'input_ids':[[0,0,0,0],[1,1,1,1]],'attention':[[0,0,0,0],[1,1,1,1]]}
# for i,j in tempdic.items():
#     print(i)
#     print(j[1])

input_ids
[1, 1, 1, 1]
attention
[1, 1, 1, 1]


In [14]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encoding, y_train)
test_dataset = CustomDataset(test_encoding, y_test)


In [16]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="epoch"
)

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


In [17]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=2)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_layer_norm.weight | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_transform.bias    | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
pre_classifier.bias     | MISSING    | 
pre_classifier.weight   | MISSING    | 
classifier.bias         | MISSING    | 
classifier.weight       | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.053320,0.022077
2,0.001255,0.034367


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=1116, training_loss=0.08193690423923795, metrics={'train_runtime': 232.5825, 'train_samples_per_second': 38.326, 'train_steps_per_second': 4.798, 'total_flos': 548894189854416.0, 'train_loss': 0.08193690423923795, 'epoch': 2.0})

In [22]:
trainer.predict(test_dataset)[1]

array([0, 0, 0, ..., 0, 0, 0])

In [23]:
prediction = trainer.predict(test_dataset)[1]

In [24]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,prediction)

array([[966,   0],
       [  0, 149]])

In [25]:
trainer.save_model('senti_model')

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
!zip -r my_colab_project.zip results senti_model sample_data

  adding: results/ (stored 0%)
  adding: results/checkpoint-500/ (stored 0%)
  adding: results/checkpoint-500/config.json (deflated 49%)
  adding: results/checkpoint-500/training_args.bin (deflated 53%)
  adding: results/checkpoint-500/model.safetensors (deflated 8%)
  adding: results/checkpoint-500/trainer_state.json (deflated 76%)
  adding: results/checkpoint-500/optimizer.pt (deflated 34%)
  adding: results/checkpoint-500/scheduler.pt (deflated 62%)
  adding: results/checkpoint-500/rng_state.pth (deflated 26%)
  adding: results/checkpoint-1000/ (stored 0%)
  adding: results/checkpoint-1000/config.json (deflated 49%)
  adding: results/checkpoint-1000/training_args.bin (deflated 53%)
  adding: results/checkpoint-1000/model.safetensors (deflated 8%)
  adding: results/checkpoint-1000/trainer_state.json (deflated 77%)
  adding: results/checkpoint-1000/optimizer.pt (deflated 34%)
  adding: results/checkpoint-1000/scheduler.pt (deflated 61%)
  adding: results/checkpoint-1000/rng_state.pth 

In [29]:
from google.colab import files
files.download("my_colab_project.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>